In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist=input_data.read_data_sets("d:/udacity-ml/public_data", one_hot=True)

Extracting d:/udacity-ml/public_data\train-images-idx3-ubyte.gz
Extracting d:/udacity-ml/public_data\train-labels-idx1-ubyte.gz
Extracting d:/udacity-ml/public_data\t10k-images-idx3-ubyte.gz
Extracting d:/udacity-ml/public_data\t10k-labels-idx1-ubyte.gz


In [8]:
mnist.train.images.shape

(55000, 784)

In [9]:
mnist.validation.images.shape

(5000, 784)

In [10]:
mnist.test.images.shape

(10000, 784)

In [11]:
mnist.train.labels.shape

(55000, 10)

In [12]:
mnist.train.next_batch(10)

(array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32),
 array([[ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]]))

In [13]:
mnist.train.images[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [14]:
mnist.train.labels[0]

array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.])

In [22]:
tf.reset_default_graph()

x=tf.placeholder(dtype=tf.float32, shape=[None, 784], name="x")
y=tf.placeholder(dtype=tf.float32, shape=[None, 10], name="y")

images=tf.reshape(x, [-1, 28, 28, 1])

with tf.name_scope("conv1"):
    # (-1, 28, 28, 1)
    conv1=tf.layers.conv2d(inputs=images, filters=32, kernel_size=[5,5], strides=[1,1], padding='same', activation=tf.nn.relu)
    # (-1, 28, 28, 32)
    pool1=tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=[2,2])
    # (-1, 14, 14, 32)

with tf.name_scope("conv2"):
    # (-1, 14, 14, 32)
    conv2=tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5,5], strides=[1,1], padding="same", activation=tf.nn.relu)
    # (-1, 14, 14, 64)
    pool2=tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=[2,2])
    # (-1, 7, 7, 64)
    
with tf.name_scope("fc"):
    # (-1, 7, 7, 64)
    flat1=tf.reshape(pool2, [-1, 7*7*64])
    # (-1, 7*7*64)
    
    fc1=tf.layers.dense(inputs=flat1, units=1024, activation=tf.nn.relu)
    # (-1, 1024)
    
    dropout1=tf.layers.dropout(inputs=fc1, rate=0.5)
    
    logits=tf.layers.dense(inputs=dropout1, units=10, activation=None)

with tf.name_scope("train"):
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    optimizer=tf.train.AdamOptimizer(learning_rate=0.001)
    train_opt=optimizer.minimize(loss)

with tf.name_scope("accuracy"):
    accuracy= tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1)), tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(100):
        batch_x, batch_y =mnist.train.next_batch(64)
        
        _,train_loss=sess.run([train_opt, loss], feed_dict={
            x:batch_x,
            y:batch_y
        })
        
        if step%20==0:
            val_accuracy=sess.run(accuracy, feed_dict={
                x:mnist.validation.images,
                y:mnist.validation.labels
            })
            #print("step:%d, train loss:%f, validation accuracy:%f" % (step, train_loss, val_accuracy))
            print("step:",step)
            print("train loss:", train_loss)
            print("validation accuracy:", val_accuracy)
    # test
    test_accuracy=sess.run(accuracy, feed_dict={
        x:mnist.test.images,
        y:mnist.test.labels
    })
    print("final test accuracy:%f" % test_accuracy)

step: 0
train loss: 2.29197
validation accuracy: 0.2816
step: 20
train loss: 0.643301
validation accuracy: 0.815
step: 40
train loss: 0.351689
validation accuracy: 0.9242
step: 60
train loss: 0.274399
validation accuracy: 0.9378
step: 80
train loss: 0.170188
validation accuracy: 0.9468
